In [ ]:
%load_ext autoreload
%autoreload 2

#%config IPCompleter.greedy=True

# Rotation curves

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import astropy.units as u
from astropy.constants import G

import snapshot_obj 
import subhalo
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(subhalo)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

## Get data

In [ ]:
sim_id = "V1_LR_fix"; snap_id = 127
snap = snapshot_obj.Snapshot(sim_id, snap_id)

In [ ]:
# Which halo:
gn = 1; sgn = 0
halo = subhalo.SubhaloInstance(snap, gn, sgn)

In [ ]:
vmax = halo.get_halo_data("Vmax") * u.cm.to(u.km)
rmax = halo.get_halo_data("VmaxRadius") * u.cm.to(u.kpc)
v1kpc = halo.get_halo_data("V1kpc") * u.cm.to(u.km)

In [ ]:
# Rotation curve:
r, v_circ = dataset_compute.compute_rotation_curve(snap, gn, sgn)
r = r * u.cm.to(u.kpc)
v_circ = v_circ * u.cm.to(u.km)

In [ ]:
fig, axes = plt.subplots()

# Set axes:
#axes.set_xscale('log')
#axes.set_yscale('log')
axes.set_xlim(0,50)
#axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$r[\mathrm{kpc}]$', fontsize=16)
axes.set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=16)

# Galaxy text:
axes.set_title('Rotation curve: GN={}, SGN={}'.format(gn,sgn))

axes.plot(r, v_circ)
axes.axhline(vmax,linestyle='dashed')
axes.axvline(rmax,linestyle='dashed')
axes.axhline(v1kpc,linestyle='dashed',c='black')
axes.axvline(1,linestyle='dashed',c='black')
    
#plt.legend()
plt.tight_layout()


# Save figure:
#
filename = 'cumulDistByVmax_evol_gn{}sgn{}_{}.png'.format(gn,sgn,sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")

## If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
plt.savefig(os.path.join(path,filename), dpi=200)